In [2]:
# installing pdfplumber to extract play 
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 51.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 63.9 MB/s eta 0:00:00:00:01


In [3]:
import pdfplumber

with pdfplumber.open("/kaggle/input/julius-caeser/julius-caesar_PDF_FolgerShakespeare.pdf") as pdf:
    print(pdf.pages[0].extract_text())


Folger Shakespeare Library
https://shakespeare.folger.edu/
Get even more from the Folger
You can get your own copy of this text to keep. Purchase a full copy
to get the text, plus explanatory notes, illustrations, and more.
Buy a copy


In [4]:
# storing all pages details in pages, so that it can be easily used to extract it
import pdfplumber
def extract_pdf(pdf_path):
    pages = []
    with pdfplumber.open(pdf_path) as pdf:
        for p in pdf.pages:
            text = p.extract_text() or ""
            pages.append(text)
    return pages
pages = extract_pdf("/kaggle/input/julius-caeser/julius-caesar_PDF_FolgerShakespeare.pdf")
print(pages[5])

Synopsis
Caesar’s assassination is just the halfway point of Julius Caesar. The
first part of the play leads to his death; the second portrays the
consequences. As the action begins, Rome prepares for Caesar’s
triumphal entrance. Brutus, Caesar’s friend and ally, fears that
Caesar will become king, destroying the republic. Cassius and others
convince Brutus to join a conspiracy to kill Caesar.
On the day of the assassination, Caesar plans to stay home at the
urging of his wife, Calphurnia. A conspirator, Decius Brutus,
persuades him to go to the Senate with the other conspirators and his
friend, Mark Antony. At the Senate, the conspirators stab Caesar to
death. Antony uses a funeral oration to turn the citizens of Rome
against them. Brutus and Cassius escape as Antony joins forces with
Octavius Caesar.
Encamped with their armies, Brutus and Cassius quarrel, then agree
to march on Antony and Octavius. In the battle which follows,
Cassius, misled by erroneous reports of loss, persuades a

In [5]:
# printing to observe how does the act doc looks like
print(pages[9])

9 Julius Caesar ACT 1. SC. 1
FTLN 0018 COBBLER Nay, I beseech you, sir, be not out with me.
FTLN 0019 Yet if you be out, sir, I can mend you.
MARULLUS
FTLN 0020 What mean’st thou by that? Mend me, thou saucy 20
FTLN 0021 fellow?
FTLN 0022 COBBLER Why, sir, cobble you.
FTLN 0023 FLAVIUS Thou art a cobbler, art thou?
FTLN 0024 COBBLER Truly, sir, all that I live by is with the
FTLN 0025 awl. I meddle with no tradesman’s matters nor 25
FTLN 0026 women’s matters, but withal I am indeed, sir, a
FTLN 0027 surgeon to old shoes: when they are in great danger,
FTLN 0028 I recover them. As proper men as ever trod upon
FTLN 0029 neat’s leather have gone upon my handiwork.
FLAVIUS
FTLN 0030 But wherefore art not in thy shop today? 30
FTLN 0031 Why dost thou lead these men about the streets?
FTLN 0032 COBBLER Truly, sir, to wear out their shoes, to
FTLN 0033 get myself into more work. But indeed, sir, we
FTLN 0034 make holiday to see Caesar and to rejoice in his
FTLN 0035 triumph. 35
MARULLUS
FTLN 

We included act, scene, speaker, text, and is_soliloquy in each chunk because each field captures a different essential part of the play’s structure.

Act and scene preserve the narrative order and location of the dialogue.

Speaker is necessary to attribute each line correctly.

Text holds the actual speech for retrieval or analysis.

is_soliloquy flags long uninterrupted speeches, which was a requirement in the prompt.

In [5]:
## Data ETL (Extract, Transform, Load) & Chunking Strategy done based on given requirements i.e, ., {"act": 2, "scene": 1,
## "speaker": "BRUTUS", "is_soliloquy": true}). and text
import re
import json



# The Folger PDF contains front-matter and index pages.
# We skip everything before page 8, where the actual play begins.
START_PAGE = 8

# Pattern to remove Folger line markers like "FTLN 1234"
FTLN_PAT = re.compile(r"FTLN\s*\d+")

# Pattern to remove isolated page/line numbers left after extraction
NUMBER_TOKEN = re.compile(r"\b\d+\b")

# Speaker lines printed on their own (e.g., "BRUTUS", "CASSIUS", "COBBLER A")
SPEAKER_LINE = re.compile(r"^[A-Z][A-Z\s\-]+$")

# Inline speaker format: "COBBLER A Why, sir..." or "CASCA Peace, ho!"
INLINE_SPEAKER = re.compile(r"([A-Z][A-Z\s\-]+)\s+(.*)")

# Act and Scene headers
ACT_PAT = re.compile(r"ACT\s+(\d+)", re.IGNORECASE)
SCENE_PAT = re.compile(r"(SCENE|SC\.)\s*(\d+)", re.IGNORECASE)


def clean_line(line):
    """Remove Folger artifacts like FTLN numbers, page numbers,
    and collapse multiple spaces into one."""
    line = FTLN_PAT.sub("", line)
    line = NUMBER_TOKEN.sub("", line)
    line = re.sub(r"\s{2,}", " ", line)
    return line.strip()


def parse_play(pages):
    """
    Parse the text of the play into structured dialogue chunks.
    Each chunk represents one speaker's continuous speech block.
    """
    chunks = []
    act = None
    scene = None
    speaker = None
    buffer = []  # stores lines spoken by the current speaker

    def flush_block():
        """Save the current collected speech into the chunks list."""
        if speaker and buffer:
            chunks.append({
                "act": act,
                "scene": scene,
                "speaker": speaker,
                "text": " ".join(buffer),
                # Simple heuristic: if a speech is long, assume soliloquy
                "is_soliloquy": len(buffer) > 5
            })

    # Iterate through the pages that contain the play
    for idx in range(START_PAGE, len(pages)):
        for raw in pages[idx].split("\n"):

            # Skip completely empty lines
            line = raw.strip()
            if not line:
                continue

            # Clean Folger artifacts
            line = clean_line(line)
            if not line:
                continue

            # ------------------------------------------------------
            # Detect ACT headers (e.g., "ACT 1")
            # ------------------------------------------------------
            if (m := ACT_PAT.search(raw)):
                flush_block()          # end previous block
                act = int(m.group(1))  # update act number
                speaker = None
                buffer = []
                continue

            # ------------------------------------------------------
            # Detect SCENE headers (e.g., "SCENE II")
            # ------------------------------------------------------
            if (m := SCENE_PAT.search(raw)):
                flush_block()
                scene = int(m.group(2))
                speaker = None
                buffer = []
                continue

            # ------------------------------------------------------
            # Detect speaker names printed alone on a line
            # Example:
            #   BRUTUS
            #   CASSIUS
            #   COBBLER A  (later cleaned)
            # ------------------------------------------------------
            if SPEAKER_LINE.match(line) and len(line.split()) <= 3:
                flush_block()
                speaker = line
                buffer = []
                continue

            # ------------------------------------------------------
            # Detect inline speaker format:
            #   "COBBLER A Why, sir..."
            #   "CASCA Peace, ho!"
            #
            # We treat names with at most 2 words as valid speakers.
            # ------------------------------------------------------
            m = INLINE_SPEAKER.match(line)
            if m and len(m.group(1).split()) <= 2:
                flush_block()

                raw_speaker = m.group(1).strip()
                spoken_text = m.group(2).strip()
                parts = raw_speaker.split()

                # ------------------------------------------------------
                # SPECIAL RULE:
                # If the speaker name ends with a single letter (e.g., "COBBLER A")
                # then that letter is actually part of the *spoken text*, not the name.
                # We move the letter into the beginning of the dialogue.
                # ------------------------------------------------------
                if len(parts) == 2 and len(parts[-1]) == 1:
                    letter = parts[-1]
                    speaker = parts[0]            # cleaned name
                    spoken_text = f"{letter} {spoken_text}"
                else:
                    speaker = raw_speaker

                buffer = [spoken_text]
                continue

            # ------------------------------------------------------
            # If none of the above matched, the line is normal dialogue.
            # Append it to the current block.
            # ------------------------------------------------------
            buffer.append(line)

    # Save the final block after the loop ends
    flush_block()
    return chunks


# -------------------------------
# Run parser + Write output file
# -------------------------------
chunks = parse_play(pages)

with open("julius_chunks.jsonl", "w", encoding="utf-8") as f:
    for c in chunks:
        f.write(json.dumps(c, ensure_ascii=False) + "\n")

print("Extracted", len(chunks), "chunks")
print("Saved to julius_chunks.jsonl")


Extracted 772 chunks
Saved to julius_chunks.jsonl


In [6]:
# to create small chunks with overlapping tokens
import json
from collections import defaultdict

def chunk_text(text, chunk_size=800, overlap=200):
    chunks = []
    start = 0
    n = len(text)

    while start < n:
        end = min(start + chunk_size, n)
        chunks.append(text[start:end])
        start += chunk_size - overlap

    return chunks


def build_rag_chunks(input_path="/kaggle/working/julius_chunks.jsonl",
                     output_path="/kaggle/working/julius_chunksafter.jsonl"):

    # Group all speeches by (act, scene)
    scenes = defaultdict(list)

    with open(input_path, "r", encoding="utf-8") as f:
        for line in f:
            entry = json.loads(line)
            key = (entry["act"], entry["scene"])
            speech = f"{entry['speaker']}: {entry['text']}\n"
            scenes[key].append(speech)

    # Combine and chunk
    out = []

    for (act, scene), speeches in scenes.items():
        # Full text of the scene
        combined_text = "".join(speeches)

        # Prepare prefix
        prefix = f"In act {act}, scene {scene}:\n"

        # Create overlapping chunks
        chunks = chunk_text(combined_text)

        # Save chunk with prefix
        for ch in chunks:
            out.append({
                "act": act,
                "scene": scene,
                "text": prefix + ch
            })

    # Write as JSONL
    with open(output_path, "w", encoding="utf-8") as f:
        for item in out:
            f.write(json.dumps(item) + "\n")

    print(f"Created {len(out)} chunks")
    print(f"Saved to {output_path}")


In [7]:
build_rag_chunks()

✅ Created 171 chunks
📌 Saved to /kaggle/working/julius_chunksafter.jsonl


# Experimentation  

In [14]:
!pip install --upgrade PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 62.0 MB/s eta 0:00:00:00:0100:01


In [15]:
import fitz

doc = fitz.open("/kaggle/input/julius-caeser/julius-caesar_PDF_FolgerShakespeare.pdf")
page = doc[0]
print(page.get_text("blocks"))

[(140.61279296875, 473.0227355957031, 397.3259582519531, 494.6368713378906, 'Get even more from the Folger\n', 0, 0), (88.80458068847656, 511.10137939453125, 444.0792541503906, 540.5215454101562, 'You can get your own copy of this text to keep. Purchase a full copy\nto get the text, plus explanatory notes, illustrations, and more.\n', 1, 0), (255.72393798828125, 601.4356079101562, 345.2554016113281, 618.20849609375, ' Buy a copy\n', 2, 0), (196.72567749023438, 398.5003356933594, 341.2175598144531, 412.906982421875, 'Folger Shakespeare Library\n', 3, 0), (189.85231018066406, 427.02593994140625, 348.0860900878906, 441.4325866699219, 'https://shakespeare.folger.edu/\n', 4, 0)]


In [19]:
# Kaggle-ready: combined PyMuPDF + pdfplumber ETL for Folger PDFs
# Designed so you can paste directly into a Kaggle notebook and run end-to-end.

# 1) Install PyMuPDF (needed for structured extraction)
!pip install --quiet --upgrade PyMuPDF

# 2) Full ETL + chunking pipeline
import fitz            # PyMuPDF
import pdfplumber
import re, json, logging, math
from typing import List, Tuple

logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")


# ------------------------------------------------------------
#                       HELPER UTILITIES
# ------------------------------------------------------------

# Remove Folger line-number markers like "FTLN 1234"
FTLN_RE = re.compile(r'\bFTLN\s*\d+\b', flags=re.IGNORECASE)

# Patterns that typically represent headers/footers in the Folger edition PDFs
HEADER_PATTERNS = [
    re.compile(r'Folger', re.IGNORECASE),
    re.compile(r'https?://', re.IGNORECASE),
    re.compile(r'^\s*Page\s*\d+\s*$', re.IGNORECASE),
    re.compile(r'^\s*\d+\s*$', re.IGNORECASE),          # standalone numbers
    re.compile(r'^\s*ACT\s+[IVXLC\d]+\b', re.IGNORECASE),
    re.compile(r'^\s*(SCENE|SC\.)\s*[IVXLC\d]+\b', re.IGNORECASE),
]


def page_blocks_reading_order(doc_page) -> List[Tuple[float, float, float, float, str]]:
    """
    Extract all text blocks from the page using PyMuPDF.
    PyMuPDF returns unsorted blocks, so we sort them manually
    to reconstruct true top-to-bottom, left-to-right reading order.
    """
    blocks = doc_page.get_text("blocks")   # Each block: (x0, y0, x1, y1, text, block_no, block_type)
    blocks = [b for b in blocks if b[4].strip()]  # keep only blocks with visible text

    # Sort first by vertical position, then by horizontal
    blocks_sorted = sorted(blocks, key=lambda b: (round(b[1]), round(b[0])))
    return blocks_sorted


def reconstruct_page_from_blocks(blocks) -> str:
    """
    Rebuild a coherent page by merging nearby blocks.
    Heuristic:
      - Blocks close in vertical space belong to the same paragraph
      - Otherwise start a new paragraph
    """
    if not blocks:
        return ""

    paragraphs = []
    cur_y = None
    cur_para = []

    for b in blocks:
        x0, y0, x1, y1, text, *_ = b

        if cur_y is None:
            cur_y = y0
            cur_para = [text.strip()]
        else:
            # If two blocks’ top positions differ only slightly, treat them as one paragraph
            if abs(y0 - cur_y) < 22:
                cur_para.append(text.strip())
            else:
                paragraphs.append(" ".join(cur_para))
                cur_para = [text.strip()]
                cur_y = y0

    # Add final paragraph
    if cur_para:
        paragraphs.append(" ".join(cur_para))

    return "\n\n".join(paragraphs)


def pdfplumber_page_text(path, pnum):
    """
    Fallback extractor for pages where PyMuPDF fails,
    e.g., pages with illustrations or stage directions.
    """
    with pdfplumber.open(path) as pdf:
        text = pdf.pages[pnum].extract_text() or ""
    return text


def remove_headers_footers(text: str) -> str:
    """
    Remove common Folger headers/footers while keeping meaningful play text.
    Also strips away any FTLN markers that sneak in.
    """
    lines = text.splitlines()
    kept = []
    for ln in lines:
        stripped = FTLN_RE.sub("", ln.strip())
        drop = any(pat.search(stripped) for pat in HEADER_PATTERNS)
        if not drop:
            kept.append(stripped)
    return "\n".join(kept)


def collapse_spaced_caps(text: str) -> str:
    """
    Fix Folger formatting like:
        'B R U T U S' -> 'BRUTUS'
    to make speaker detection robust.
    """
    return re.sub(r'\b(?:[A-Z]\s+){2,}[A-Z]\b',
                  lambda m: m.group(0).replace(" ", ""), text)


def normalize_hyphens_and_newlines(text: str) -> str:
    """
    Clean up visual hyphens and messy line breaks.
    This makes the downstream parsing far more stable.
    """
    text = text.replace("\u2011", "-")  # non-breaking hyphen → normal hyphen

    # Join broken hyphenated lines: "be-\n gin"  → "begin"
    text = re.sub(r'(\w)-\s*\n\s*(\w)', r'\1\2', text)

    # Reduce overly large gaps
    text = re.sub(r'\n{3,}', '\n\n', text)

    # Convert single newlines inside regular sentences into spaces
    text = re.sub(r'(?<=[a-z,;:])\n(?=[a-zA-Z0-9])', ' ', text)

    return text.strip()


# ------------------------------------------------------------
#      FIND WHERE THE REAL PLAY STARTS (ignore front matter)
# ------------------------------------------------------------

def find_play_start(full_text: str) -> int:
    """
    Folger PDFs contain front-matter, license info, TOC, etc.
    This function finds the first ACT header to mark the actual play start.
    """
    patterns = [
        re.compile(r'\n\s*ACT\s+I\b', re.IGNORECASE),
        re.compile(r'\n\s*ACT\s+1\b', re.IGNORECASE),
        re.compile(r'\n\s*A C T\s+I\b', re.IGNORECASE),  # weird spaced-out variant
        re.compile(r'\n\s*ACT\s+[IVXLC]+\b', re.IGNORECASE),
    ]

    for pat in patterns:
        m = pat.search(full_text)
        if m:
            return m.start()

    # Very rare fallback
    m = re.search(r'\n\s*SCENE\s+1\b', full_text, re.IGNORECASE)
    return m.start() if m else 0


# ------------------------------------------------------------
#          SPLIT CLEAN PLAY TEXT INTO STRUCTURED PARTS
# ------------------------------------------------------------

def split_into_acts(text: str):
    """
    Split the cleaned full text into acts.
    Returns a list of dicts: {act: "I", text: "..."}
    """
    parts = re.split(r'\bACT\s+([IVXLC]+)\b', text, flags=re.IGNORECASE)
    if len(parts) <= 1:
        return [{'act': 'I', 'text': text}]  # fallback for malformed PDFs

    acts = []
    it = iter(parts[1:])
    for label, content in zip(it, it):
        acts.append({'act': label.strip(), 'text': content})
    return acts


def split_act_into_scenes(act_text: str):
    """
    Split each act into scenes using SCENE <roman numeral>.
    """
    parts = re.split(r'\bSCENE\s+([IVXLC]+)\b', act_text, flags=re.IGNORECASE)
    if len(parts) <= 1:
        return [{'scene': '1', 'text': act_text}]

    scenes = []
    it = iter(parts[1:])
    for label, content in zip(it, it):
        scenes.append({'scene': label.strip(), 'text': content})
    return scenes


def split_scene_into_speeches(scene_text: str):
    """
    Split a scene into speech blocks using uppercase speaker names.
    This part is intentionally forgiving because Folger formatting varies.
    """
    st = FTLN_RE.sub("", scene_text)

    # Normalize big gaps
    st = re.sub(r'\n{2,}', '\n\n', st).strip()

    # Speaker-line pattern: all-caps, maybe ends in ':' or '.'
    parts = re.split(r'\n([A-Z][A-Z0-9\-\s]{1,60})(?:\.|:)?\n', st)
    if len(parts) <= 1:
        return [{'speaker': None, 'text': st.strip()}]

    speeches = []
    prefix = parts[0].strip()
    if prefix:
        speeches.append({'speaker': None, 'text': prefix})

    # Extract alternating (speaker → body) pairs
    for i in range(1, len(parts), 2):
        speaker = parts[i].strip()
        body = parts[i+1].strip() if (i+1) < len(parts) else ""
        speeches.append({'speaker': speaker, 'text': body})

    return speeches


def merge_adjacent_speeches(speeches):
    """
    If the same speaker appears multiple times consecutively (common in PDFs),
    merge them into a single speech block.
    """
    if not speeches:
        return []

    merged = []
    cur = speeches[0].copy()

    for s in speeches[1:]:
        if cur.get('speaker') == s.get('speaker') and cur.get('speaker'):
            cur['text'] = cur['text'].rstrip() + "\n" + s.get('text', '').lstrip()
        else:
            merged.append(cur)
            cur = s.copy()

    merged.append(cur)
    return merged


def is_soliloquy(speaker, text):
    """
    Lightweight heuristic for soliloquies:
      - Very long speech
      - Contains 'alone'
      - Speaker line explicitly mentions 'soliloquy'
    """
    if not text:
        return False
    if 'soliloquy' in (speaker or '').lower():
        return True
    if len(text.split()) > 80:
        return True
    if re.search(r'\balone\b', text, flags=re.IGNORECASE):
        return True
    return False


# ------------------------------------------------------------
#                  FULL PAGE → CLEAN TEXT ETL
# ------------------------------------------------------------

def etl_combined(pdf_path: str) -> str:
    """
    Combined text extractor:
        1) PyMuPDF for layout-aware block extraction
        2) pdfplumber fallback for problematic pages
        3) Heavy cleanup: remove headers, fix spacing, collapse caps, etc.
    Returns a single clean text string containing only the play.
    """
    doc = fitz.open(pdf_path)
    pages_text = []

    for pnum in range(len(doc)):
        page = doc[pnum]

        # Step 1: reconstruct from positional text blocks
        blocks = page_blocks_reading_order(page)
        page_text = reconstruct_page_from_blocks(blocks)

        # Step 2: fallback if PyMuPDF yields suspiciously little text
        if len(page_text.strip()) < 50:
            try:
                page_text = pdfplumber_page_text(pdf_path, pnum)
            except Exception:
                page_text = ""

        # Step 3: remove headers/footers and tidy format
        page_text = remove_headers_footers(page_text)
        page_text = normalize_hyphens_and_newlines(page_text)

        pages_text.append(page_text)

    full = "\n\n".join(pages_text)

    # Fix spaced-out names before parsing
    full = collapse_spaced_caps(full)

    # Trim away front matter → jump to first ACT
    start_idx = find_play_start(full)
    if start_idx > 0:
        full = full[start_idx:]

    # Final cleanup
    full = FTLN_RE.sub("", full)
    full = re.sub(r'\n{3,}', '\n\n', full).strip()

    return full


# ------------------------------------------------------------
#                   CLEAN TEXT → JSONL CHUNKS
# ------------------------------------------------------------

def parse_to_chunks(full_text: str):
    """
    Convert cleaned full play text into structured JSONL-ready chunks:
      - act
      - scene
      - speaker
      - is_soliloquy
      - text
    """
    acts = split_into_acts(full_text)
    chunks = []

    for act_entry in acts:
        act_label = act_entry['act']
        scenes = split_act_into_scenes(act_entry['text'])

        for scene_entry in scenes:
            scene_label = scene_entry['scene']
            speeches = split_scene_into_speeches(scene_entry['text'])
            speeches = merge_adjacent_speeches(speeches)

            for sp in speeches:
                speaker = sp.get('speaker') or "UNKNOWN"
                text = sp.get('text', '').strip()
                if not text:
                    continue

                chunks.append({
                    "act": act_label,
                    "scene": scene_label,
                    "speaker": speaker,
                    "is_soliloquy": is_soliloquy(speaker, text),
                    "text": text,
                    "source": f"Act {act_label} Scene {scene_label} - {speaker}"
                })

    return chunks


def write_jsonl(chunks, out_path):
    """Write list of chunk dicts to a JSONL file."""
    with open(out_path, "w", encoding="utf-8") as f:
        for c in chunks:
            f.write(json.dumps(c, ensure_ascii=False) + "\n")


# ------------------------------------------------------------
#                     KAGGLE ENTRYPOINT
# ------------------------------------------------------------

def main(pdf_path, out_jsonl):
    """
    Full pipeline runner:
       → Read & clean PDF
       → Parse into acts/scenes/speeches
       → Write to JSONL
    """
    logging.info("Running combined ETL (PyMuPDF + pdfplumber)...")
    full = etl_combined(pdf_path)

    logging.info("Parsing into chunks...")
    chunks = parse_to_chunks(full)

    logging.info(f"Chunks produced: {len(chunks)}")
    write_jsonl(chunks, out_jsonl)
    logging.info(f"Wrote JSONL to {out_jsonl}")

    return chunks



In [20]:
chunks = main(
    "/kaggle/input/julius-caeser/julius-caesar_PDF_FolgerShakespeare.pdf",
    "/kaggle/working/julius_chunks.jsonl"
)

len(chunks)

INFO: Running combined ETL (PyMuPDF + pdfplumber)...
INFO: Parsing into chunks...
INFO: Chunks produced: 60
INFO: Wrote JSONL to /kaggle/working/julius_chunks.jsonl


60

In [23]:
chunks[3]

{'act': 'I',
 'scene': '1',
 'speaker': 'POPILIUS LENA',
 'is_soliloquy': False,
 'text': 'tribunes FLAVIUS MARULLUS\n\nMARK ANTONY rulers of Rome in Acts 4 and 5 LEPIDUS\n\nOCTAVIUS Servant to Antony Servant to Octavius',
 'source': 'Act I Scene 1 - POPILIUS LENA'}